In [1]:
spark

In [6]:
df = (spark.read
     .format("com.mongodb.spark.sql.DefaultSource")
     .option("uri", "mongodb://localhost/bakery.sample")
     .load())

In [7]:
df.createOrReplaceTempView('reviews')

In [25]:
query='''
    SELECT COUNT(`제목`) cnt, `닉네임` id FROM reviews
    GROUP BY 2
    ORDER BY cnt DESC
'''
spark.sql(query).show(10)

+---+-----------+
|cnt|     닉네임|
+---+-----------+
| 79|     Bella |
| 44| 림 윤이맘 |
| 44|  보리새싹 |
| 43|    김빵돌 |
| 41|    루시에 |
| 35|  막강쯔지 |
| 32|  기믄슈기 |
| 29|  곰돌이존 |
| 27|  하투하투 |
| 26|   gomdong |
+---+-----------+
only showing top 10 rows



In [33]:
query='''
    SELECT `작성날짜` time, `내용` text
    FROM reviews
'''
reviews_table = spark.sql(query)

In [34]:
from pyspark.sql import Row

def text_split(row):
    for word in row.text.split():
        yield Row(time=row.time, word=word)

In [35]:
reviews_table.rdd.take(1)

[Row(time='2020-07-27T12:16:00', text='   후기 매장 명 코코로카라 바나나오레오 카라벨라다크푸딩에 이어 바나나 미쯔말차 발로나밀크 먹어본 후기 가져왔어요   바나나푸딩 바나나의 부드럽고 달콤함 닐라웨이퍼의 촉촉 달달 고소함 생과일바나나의 신선함 까지 합해져서 찰떡궁합이에요 바나나 좋아 는 분은 프로 좋아하실맛익숙하지만 특별해요 바나나 덕후분들 메모메모 발로나밀크푸딩 카라벨라다크푸딩과는 또다른 매력이있는 푸딩이에요밀크초코지만  보다 엄청 달지않은 점이 더 맘에 들었어요 그렇다고 안 달다고는 안했습니다   초코의 풍미가 엄청 잘 느껴지고 부재료 랑 궁합도 너무 좋아요 다크푸딩과 마찬가지로 오렌지필들어있구요중간중간 상큼함 오렌지가 씹혀서 질리지않고 더 맛있게 먹을 수 있었어요 저는 신기하게도 다크보다 밀크가 더 술술 들어가더라구요 초코 좋아 면 그냥 드셔주세요 미쯔말차푸딩 제가 어린이입맛에 가까워서 말차 특유의 쌉쌀함이 너무 강하면 잘 못먹는편이라 살 때 고민을 가장 많이 했던 푸딩이었어요 근데 사길 정말 잘했다는  이 드는 푸딩 지금까지 먹어본 코코로푸딩중에선 이게 등이었구요아주 진한 말차를 선호 는 분들은 조금 약하다고 아쉬워하실 수 있겠지만 전 딱 좋은 정도였어요엄청 달지않고 부드러우면서도 말차의 매력은 살아있는 약간 말차라떼느낌 거기에 미쯔랑 초콜릿까지 오독오독 씹히니까   맛있어서 미치겠더라구요 ㅠㅠㅠ 저 수요일에 또 갈 예정인데   사러 가는게 가장 큰 이유에요녹린이인 저도 반할만큼 맛있고 대중적인 말차맛이니까 저같은 분들 꼭 드셔보셨음 좋겠어요 저희집에 코코로카라가 가까운 거리는 아니에요하지만 그럼에도 불구하고 계속 가게되는 곳이구 제 마음속 푸딩 맛집 등입니다 다들 꼭 드셔보셨음 좋겠네요 ')]

In [37]:
reviews_table.rdd.flatMap(text_split).take(10)

[Row(time='2020-07-27T12:16:00', word='후기'),
 Row(time='2020-07-27T12:16:00', word='매장'),
 Row(time='2020-07-27T12:16:00', word='명'),
 Row(time='2020-07-27T12:16:00', word='코코로카라'),
 Row(time='2020-07-27T12:16:00', word='바나나오레오'),
 Row(time='2020-07-27T12:16:00', word='카라벨라다크푸딩에'),
 Row(time='2020-07-27T12:16:00', word='이어'),
 Row(time='2020-07-27T12:16:00', word='바나나'),
 Row(time='2020-07-27T12:16:00', word='미쯔말차'),
 Row(time='2020-07-27T12:16:00', word='발로나밀크')]

In [38]:
words = reviews_table.rdd.flatMap(text_split).toDF()
words.createOrReplaceTempView('words')

In [39]:
query='''
    SELECT word, count(*) count
    FROM words
    GROUP BY 1
    ORDER BY 2 DESC
'''
spark.sql(query).show(10)

+------+--------+
|  word|count(1)|
+------+--------+
|  너무|    2953|
|    더|    2243|
|    빵|    2163|
|    다|    1845|
|    것|    1827|
|  많이|    1826|
|    좀|    1698|
|같아요|    1589|
|  정말|    1562|
|    잘|    1486|
+------+--------+
only showing top 10 rows

